# 加载基本库

In [29]:
import collections
import pandas as pd
import datetime
import ffmyh
import time
import math
# from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn.metrics import log_loss
import xgboost
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
import sklearn
from sklearn.feature_extraction import DictVectorizer


# from fastFM.datasets import make_user_item_regression
# from fastFM import als
from sklearn.metrics import mean_squared_error
import scipy.sparse as sp
import numpy as np

sklearn.__version__

'0.19.0'

# 加载数据项

In [2]:
train_df_org = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_train_20180301.txt',sep=' ')
test_df_org = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_test_a_20180301.txt',sep=' ')



In [30]:
train_df_org.columns

train_df = train_df_org.copy().set_index("context_id")
test_df = test_df_org.copy().set_index("context_id")

test_df["is_trade"] = -1
train_df = train_df.append(test_df)
# print len(train_df)
# print train_df[["is_trade"]]

# 预处理数据

In [ ]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数
# date最终不使用，直接用day(第 0 - 7 天)

from datetime import datetime

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

train_df['date'] = train_df['context_timestamp'].apply(lambda x: extract_date(x))
train_df['weekday'] = train_df['context_timestamp'].apply(lambda x: extract_weekday(x))
train_df['hour'] = train_df['context_timestamp'].apply(lambda x: extract_hour(x))

m = {}
for idx, date in enumerate(sorted(train_df['date'].unique())):
    m[date]=idx
print(m)

for idx, d in enumerate(sorted(train_df['date'].unique())):
    train_df["day"] = train_df['date'].map(lambda x: m[x])

{'2018-09-24': 6, '2018-09-25': 7, '2018-09-20': 2, '2018-09-21': 3, '2018-09-22': 4, '2018-09-23': 5, '2018-09-19': 1, '2018-09-18': 0}


# 拆解复杂类型

In [ ]:
#简化list等复杂类型的结构
#item_category_list全展开
#item_property_list全展开取频率>0.05的数据
#predict_category_property会计算跟item prop和cate的余弦相似度
#具体是把两个list的数据拼成cate:-1和cate:prop两种方式拼成一个字符串再跟predict_category_property的数据计算相似度
def get_icl_map(df):
    print("get_icl_map ... ")
    dfX = df.copy()
    dfX = dfX['item_category_list'].str.split(';', expand=True)

#     s = sorted(list(dfX[0].unique()) + list(dfX[1].unique()) + list(dfX[2].unique()))
    m = {}
    for i in dfX[0].unique():
        if i == None:
            continue
        m[i] = "1"
    
    for i in dfX[1].unique():
        if i == None:
            continue
        m[i] = "2"
    
    for i in dfX[2].unique():
        if i == None:
            continue
        m[i] = "3"
    return m

def get_ipl_map(df):
    print("get_ipl_map ... ")
    df1 = df.copy()
    dfX = df1.copy()['item_property_list'].str.split(';')
    dfX = pd.DataFrame(dfX)
    
    m = collections.defaultdict(float)
    idx = 0
    for _, row in dfX.iterrows():
        for i in row[0]:
            m[i] += 1
    
    ll = len(dfX)
    for k,v in m.items():
        m[k] = v / ll
    return m

def process_complex_types(dfX, icl_map, ipl_map):
    def filter_unless_cate(arr):
        ret = []
        for i in arr:
            if i in icl_map:
                ret.append(i)
        if len(ret) == 0:
            return None
        else:
            return ret
    
    def filter_unless_prop(arr):
        ret = []
        for i in arr:
            freq = ipl_map.get(i, 0.)
            if freq > 0.05:
                ret.append(i)
        if len(ret) == 0:
            return None
        else:
            return ret
    
    def unique_list(arr):
        return list(set(arr))
    
    #{cate}:-1命中则为1分
    #{cate}:{prop}命中则为2分
    #后期优化权重
    def inner_product_recall_items(line):
        line = line.split("|")
        item_category_list = unique_list(line[0].split(";"))
        item_property_list = unique_list(line[1].split(";"))
        
        whole_combines = {}
        for cate in item_category_list:
            tmp = cate+":"+"-1"
            whole_combines[tmp] = 1
            for prop in item_property_list:
                tmp = cate+":"+prop
                whole_combines[tmp] = 2
        
        
                
        predict_category_property = unique_list(line[2].split(";"))
        product = 0.
        item_vec_len = math.sqrt(len(whole_combines))
        user_vec_len = math.sqrt(len(predict_category_property))
        for item in predict_category_property:
            #x1 == 1
            #y1 == 1
            #x1*y1 == 1
            #x2 == 0
            #y2 == 1
            #x2*y2 == 0
            #所以product由x决定 += 1/0
            product += whole_combines.get(item, 0)
        
        return product/(item_vec_len*user_vec_len)
            
        
            
        
        
        
    print("processing predict_category_property ...")
#     dfX['predict_category_property'] = dfX['predict_category_property'].str.split(';').map(lambda x: [i.split(":")[0] for i in x]).map(filter_unless_cate)
    
    dfX['predict_richness'] =  dfX['predict_category_property'].map(lambda x: 0 if len(x.strip()) == 0 else len(x.split(";")))
    dfX['predict_category_property'] = dfX['item_category_list']+"|"+dfX['item_property_list']+"|"+dfX['predict_category_property']
    dfX['predict_category_property'] = dfX['predict_category_property'].map(inner_product_recall_items)
    
    print("processing item_property_list ...")
    dfX['item_property_richness'] = dfX['item_property_list'].map(lambda x: 0 if len(x.strip()) == 0 else len(x.split(";")))
    dfX['item_property_list'] = dfX['item_property_list'].str.split(';').map(filter_unless_prop).map(unique_list)
    
    print("processing item_category_list ...")
    dfX['item_category_list'] = dfX['item_category_list'].str.split(';')
    
    print "generating item_category_1, item_category_2 ..."
#     dfX['item_category_list01'] = dfX['item_category_list'].map(lambda x:x[0] if x != None and len(x) > 0 else None)
    dfX['item_category_1'] = dfX['item_category_list'].map(lambda x:x[1] if x != None and len(x) > 1 else None)
    dfX['item_category_2'] = dfX['item_category_list'].map(lambda x:x[2] if x != None and len(x) > 2 else None)
    
    return dfX



# aaa = process_complex_types(train_df.copy(), get_icl_map(train_df), get_ipl_map(train_df))

train_df = process_complex_types(train_df, get_icl_map(train_df), get_ipl_map(train_df))


get_icl_map ... 
get_ipl_map ... 


# 转化属性增加trade_rate + trade_pv（按前一天）

In [ ]:
# 统计信息附加

def calcTVTransform(df, key, key_y, filter_src, filter_dst, smoothing = 5, mean0=None):
    if mean0 is None:
        #计算目标的平均值做平缓用
        mean0 = df.ix[filter_src, key_y].mean()
        print("mean0:", mean0)
    
    #取出key的所有值
    df['_key1'] = df[key].astype('category').values.codes
    
    
    #取出用于计算的源（后面聚合掉就没有顺序可言了）
    df_key1_y = df.ix[filter_src, ['_key1', key_y]]
    
    #根据key的取值去聚合key_y的总数和总和，用户计算rate和count
    grp1 = df_key1_y.groupby(['_key1'])
    sum1 = grp1[key_y].aggregate(np.sum)
    cnt1 = grp1[key_y].aggregate(np.size)
    
    vn_sum = 'sum_' + key
    vn_cnt = 'cnt_' + key
    
    #取出dst（带序列）的所有key
    v_codes = df.ix[filter_dst, '_key1']
    
    #得到_sum,_cnt，按dst的序列
    _sum = sum1[v_codes].values
    _cnt = cnt1[v_codes].values
    _cnt[np.isnan(_sum)] = 0    
    _sum[np.isnan(_sum)] = 0
    
    r = {}
    r['exp'] = (_sum + smoothing * mean0)/(_cnt + smoothing)
    r['cnt'] = _cnt
    return r


In [ ]:
#计算前一天的交易率set到下一天，第0天用回自己
# tmp = train_df.copy()
tmp = train_df

add_count = False
# window = 2

exp = "exp_d_"
cnt = "cnt_d_"


exp_numerical = {}
cnt_numerical = {}

In [ ]:
tmp = train_df
k = "day"
exp_k = "exp_d_day"
for day in xrange(0,7):
    cal_day = day
    set_day = day

    print("cal %s trade_rate cnt %s set to %s" % (k,cal_day, set_day))

    #start_d - day(不含day)用于计算，结果赋值到day上
    days1 = (tmp.day.values == cal_day)
    days2 = (tmp.day.values == set_day)
    ret = calcTVTransform(tmp, k, 'is_trade', days1, days2, 0)

    tmp.loc[tmp.day.values == day, exp_k] = ret["exp"]

    exp_numerical[exp_k]=1

tmp.loc[tmp.day.values == 7, exp_k] = 0.0169

tmp[[exp_k, "day"]]

In [ ]:


#此处应该处理按天为梯度的数据
for k in ['item_price_level',
'item_sales_level',
'item_collected_level',
'item_pv_level',
'user_gender_id',
'user_age_level',
'user_occupation_id',
'user_star_level',
'context_page_id',
'shop_review_num_level',
'shop_star_level',
         

"item_brand_id",
"item_city_id",
"item_id",
"shop_id"
         ]:
    exp_k = exp+k
    cnt_k = cnt+k
    for day in xrange(0,8):
#         start_d = max(day - window, 0)
#         end_d = max(day - 1,0)
        cal_day = max(day - 1, 0)
        set_day = day

        print("cal %s trade_rate cnt %s set to %s" % (k,cal_day, set_day))
        
        #start_d - day(不含day)用于计算，结果赋值到day上
        days1 = (tmp.day.values == cal_day)
        days2 = (tmp.day.values == set_day)
        ret = calcTVTransform(tmp, k, 'is_trade', days1, days2)
            
        tmp.loc[tmp.day.values == day, exp_k] = ret["exp"]
        
        exp_numerical[exp_k]=1
        if add_count:
            cnt_numerical[cnt_k]=1
            tmp.loc[tmp.day.values == day, cnt_k] = ret["cnt"]


In [ ]:
exp_numerical = exp_numerical.keys()
cnt_numerical = cnt_numerical.keys()

print(exp_numerical)
print(cnt_numerical)

# 标记训练用的列

In [ ]:
#标记训练数据
%run FeatureProcess.py
import copy
target="is_trade"
                          
                          
                        
categorical=[  
                'item_price_level',
                'item_sales_level',
                'item_collected_level',
                'item_pv_level',
                'user_gender_id',
                'user_age_level',
                'user_occupation_id',
                'user_star_level',
                'context_page_id',
                'shop_review_num_level',
                'shop_star_level',
                "weekday",
                "hour",
    
                'item_category_1',
                'item_category_2',
                
                "item_brand_id",
                "item_city_id",
                "item_id",
                "shop_id"
            ]

numerical=[     'shop_review_positive_rate',
                'shop_score_service',
                'shop_score_delivery',
                'shop_score_description',
                'predict_category_property',
                'predict_richness',
                'item_property_richness'
                   
#               ]
          ]+exp_numerical

listype = [
    'item_property_list', 
#     'item_category_list'
]

                    
            

cols_tool = filter_on_cols(target, categorical, numerical, listype)

In [ ]:
train_df[cols_tool.get_raw_numerical_cols()].describe()

# 平滑处理连续型数据最后对复杂类型做onehot编码

In [20]:
#数据的预处理
#double系列(例如好评率)的数据存在一个问题，没有考虑到评论量，所以乘了一个评论level，类似 好评率*评论数(分桶了) = 好评量(分桶了)
#fillempty把-1设置成-0.01，配合mov2pos使用
#mov2pos把有<0的数据都集体-min，如果一个数据是0-1，有-1出现的时候，上面设置成了-0.01，所以这列数据真实是-0.01 - 1,然后我会集体-(-0.01)
#norm用了最大值最小值norm
%run FeatureProcess.py
from sklearn import preprocessing

def bucketizer(df, key, bins):
    le = preprocessing.LabelEncoder()
    prefix="*BUK*_"
    df[prefix+key] = pd.cut(df[key].values, bins=5, retbins=False)
    df[prefix+key] = le.fit_transform(df[prefix+key].copy())
    return df


def layering(df, key, by_key):
    uniqv = df[by_key].unique()
    keys = []
    for v in uniqv:
        indexer = (df[by_key] == v)
        new_key = "*%sLAY*_%s" % (str(v), key)
        df.loc[indexer, new_key] = df[indexer][key]
        keys.append(new_key)
    return df, keys

# #分桶+打散
# print("bucketizer ... ")
# train_df = bucketizer(train_df, "shop_review_num_level", 5)

# print("layering ... ")
# new_n = []
# for n in numerical:
#     train_df, keys = layering(train_df, n, '*BUK*_shop_review_num_level')
#     new_n += keys



featProc = FeatureProcess(target=target, categorical=categorical, numerical=numerical, listype = listype)
# for n in numerical:
#     featProc.addLayeringOrgKeys(n)
# train_df["shop_review_num_level"] = train_df["shop_review_num_level"]
# train_df["shop_review_positive_rate"] = train_df["shop_review_positive_rate"]*train_df["shop_review_num_level"]
# train_df["shop_score_service"] = train_df["shop_score_service"]*train_df["shop_review_num_level"]
# train_df["shop_score_delivery"] = train_df["shop_score_delivery"]*train_df["shop_review_num_level"]
# train_df["shop_score_description"] = train_df["shop_score_description"]*train_df["shop_review_num_level"]

train_df = featProc.fillempty(train_df, -0.01)
# train_df = featProc.mov2pos(train_df)
train_df = featProc.mov2mean(train_df)
train_df = featProc.norm(train_df)
train_df[cols_tool.get_raw_numerical_cols()].describe()


,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description,predict_category_property,predict_richness,item_property_richness,exp_d_user_age_level,exp_d_item_pv_level,exp_d_shop_review_num_level,exp_d_item_sales_level,exp_d_user_occupation_id,exp_d_item_city_id,exp_d_item_brand_id,exp_d_user_star_level,exp_d_item_price_level,exp_d_day,exp_d_item_id,exp_d_shop_star_level,exp_d_item_collected_level,exp_d_user_gender_id,exp_d_context_page_id,exp_d_shop_id
count,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000
mean,0.994898,0.971535,0.970906,0.975237,0.164666,0.293092,0.302926,0.688612,0.101611,0.218402,0.511220,0.836325,0.098886,0.081723,0.341940,0.166898,0.606917,0.064086,0.153373,0.538369,0.699759,0.687828,0.071759
std,0.009858,0.014145,0.014084,0.016028,0.117458,0.138144,0.107984,0.115209,0.014680,0.044658,0.162354,0.080191,0.042041,0.074096,0.052908,0.076621,0.361303,0.077242,0.031012,0.086564,0.145956,0.124316,0.079743
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.992852,0.966693,0.966017,0.969586,0.083333,0.153846,0.244681,0.626489,0.094769,0.200471,0.399855,0.812573,0.070562,0.028698,0.326793,0.121504,0.120499,0.012521,0.132899,0.500887,0.674225,0.666383,0.015845
50%,1.000000,0.972620,0.971860,0.978705,0.156556,0.307692,0.287234,0.684715,0.102018,0.214012,0.513787,0.828809,0.087382,0.076188,0.348955,0.141488,0.772863,0.038448,0.151369,0.537175,0.714830,0.695428,0.048508
75%,1.000000,0.978041,0.977194,0.983789,0.235412,0.384615,0.351064,0.748742,0.106222,0.247507,0.588760,0.866525,0.117639,0.106783,0.373371,0.205879,0.891105,0.085545,0.180926,0.576776,0.745833,0.747943,0.103168
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
#onehot没啥好说的，就是onehot，toOneHotList实现有点不同分开了而已


train_df = featProc.toOneHot(train_df)
train_df = featProc.toOneHotList(train_df)

# train_df = featProc.cacheRun(featProc.toOneHot, train_df)

for c in train_df.columns:
    print(c)

print train_df.columns

toOneHot ...
toOneHotList ...


# 定义模型函数

In [21]:
%run ffmyh.py

    
# def train_and_test_combine(train_df, test_df, res_df):
    
#     X_train = train_df.copy()
#     del X_train['is_trade']
#     y_train = train_df['is_trade']
    
#     X_test = test_df.copy()
#     del X_test['is_trade']
#     y_test = test_df['is_trade']
    
    
    
#     model = xgboost.XGBClassifier(nthread=7,max_depth=5)
#     model.fit(X_train, y_train)
    
#     y_prev_predict_train = pd.DataFrame(model.predict_proba(X_train))
#     y_prev_predict_train["idx"] = X_train.index
#     y_prev_predict_train = y_prev_predict_train.set_index("idx")
#     X_train["y_prev_predict"] = y_prev_predict_train[1]
    
#     y_prev_predict_test = pd.DataFrame(model.predict_proba(X_test))
#     y_prev_predict_test["idx"] = X_test.index
#     y_prev_predict_test = y_prev_predict_test.set_index("idx")
#     X_test["y_prev_predict"] = y_prev_predict_test[1]
    
    
# #     X_train = X_train.fillna(0)
# #     X_test = X_test.fillna(0)
    
    
#     model = LogisticRegression(max_iter=1000)
#     model.fit(X_train, y_train)
    
    
#     train_lls = log_loss(y_train,model.predict_proba(X_train))
#     test_lls = log_loss(y_test,model.predict_proba(X_test))
#     del train_df
#     del test_df
    
#     del X_train
#     del y_train
#     del X_test
#     del y_test
    
#     return train_lls, test_lls
    

def train_and_test_xgboost(train_df, test_df, res_df):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
#     model = xgboost.XGBClassifier(reg_lambda=1.5,learning_rate=0.1,reg_alpha=0,nthread=7,n_estimators=100,max_depth=10)
    model = xgboost.XGBClassifier(subsample=0.9,colsample_bytree=0.9,n_estimators=500,max_depth=7,nthread=7)

#     model = xgboost.XGBClassifier(subsample=0.9,colsample_bytree=0.9,n_estimators=50,max_depth=3,nthread=7)
    model.fit(X_train, y_train)
    fig, ax = plt.subplots(figsize=(12,18))
    xgboost.plot_importance(model,ax=ax)
    plt.show()
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("xgboost_res.csv",index=False,sep=' ')
        
    
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_lr(train_df, test_df, res_df):    
    X_train = train_df.copy().fillna(0)
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy().fillna(0)
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
    
    model = LogisticRegression(C=1.0, n_jobs=7,tol=1e-6, max_iter=2000)
    model.fit(X_train.values, y_train)
#     for idx, val in enumerate(list(X_train.columns)):
#         print("%s=%s" %(val,list(model.coef_[0])[idx]))


    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        res_df = res_df.copy().fillna(0)
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("lr_res.csv",index=False,sep=' ')
    
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_randomforest(train_df, test_df, res_df):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
#     rf = RandomForestClassifier(n_estimators=32, max_depth=40, min_samples_split=100, min_samples_leaf=10,  criterion='entropy',
#                         max_features=8, verbose = 1,  bootstrap=False, n_jobs=10)
#     RandomForestClassifier(criterion='entropy',n_estimators=100,n_jobs=15)

    model = RandomForestClassifier(n_jobs=7, n_estimators=100, max_depth=10, min_samples_split=100, min_samples_leaf=10,  criterion='entropy', max_features=8, verbose = 1,  bootstrap=False)
    model.fit(X_train, y_train)
    
    ft_w = pd.DataFrame(model.feature_importances_, columns=["weights"], index = list(X_train.columns))
    print ft_w.sort_values("weights", ascending=False)
#     fig, ax = plt.subplots(figsize=(12,18))
#     xgboost.plot_importance(model,ax=ax)
#     plt.show()
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("randomforest_res.csv",index=False,sep=' ')
        
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_gbdt(train_df, test_df, res_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    model = GradientBoostingClassifier(n_estimators=100, max_depth=10, min_samples_split=100, min_samples_leaf=10, max_features=4)
    model.fit(X_train, y_train)
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("gbdt_res.csv",index=False,sep=' ')
        
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls


def train_and_test_lgb(train_df, test_df, res_df, target, features, categorical):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
    
    
#     test['lgb_predict'] = clf.predict_proba(test[features],)[:, 1]
#     print(log_loss(test[target], test['lgb_predict']))

    
    model = lgb.LGBMClassifier(num_leaves=10, max_depth=7, n_estimators=80, n_jobs=20)
    model.fit(X_train, y_train, feature_name=features,categorical_feature=categorical)
    
    ft_w = pd.DataFrame(model.feature_importances_, columns=["weights"], index = list(X_train.columns))
    print ft_w
#     fig, ax = plt.subplots(figsize=(12,18))
#     xgboost.plot_importance(model,ax=ax)
#     plt.show()
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("lgb_res.csv",index=False,sep=' ')
        
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls


def train_and_test_FFM(train_df, test_df, res_df):
    global cols_tool
    
    model = FFM(target=cols_tool.target, categorical=cols_tool.categorical, numerical=cols_tool.numerical, listype=cols_tool.listype,
                                        reg_param = 0.0004,
                                        k = 4,
                                        iter_max = 500,
                                        learing_rate = 0.1,
                                        threads = 7,
                                        auto_stop = False,
                                        quiet = False,
                                        no_norm = False)


    
    model.fit(train_df, test_df)
    
    train_lls = log_loss(train_df["is_trade"],model.predict_proba(train_df))
    test_lls = log_loss(test_df["is_trade"],model.predict_proba(test_df))
    
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("ffm.csv",index=False,sep=' ')
        
    
    return train_lls, test_lls


def select_best_features(df, precent=0.8):
    from sklearn.feature_selection import SelectKBest,chi2 
    df = df.copy()
    
    for u in df.columns:
        df[df[u] == -1] = 0

    X_train = df.copy()
    del X_train['is_trade']
    y_train = df['is_trade']
    
    c = int(len(X_train.columns) * precent)
    skb = SelectKBest(chi2,k=c)
    skb.fit_transform(X_train,y_train)
    
    old_fea = list(X_train.columns)
    new_fea = []
    for idx, b in enumerate(list(skb.get_support())):
        if b:
            new_fea.append(old_fea[idx])
        else:
            print("Filter feature:%s" % old_fea[idx])
    return new_fea

# 开搞！

In [27]:
#这块一般不怎么需要改了，拿最后一天做验证
#注意下面验证版和测试版，测试版是我们自己玩的，验证版是扔线上的，通过注释不同的跑就行了


# %run BaseFrame.py
% run ../util/time_series_split.py

tmp_train_df = train_df.copy()#.fillna(-1)

# tmp_train_df["user_gender_id*shop_review_positive_rate"] = tmp_train_df["user_gender_id"] * (1+tmp_train_df["shop_review_positive_rate"])

# tmp_train_df["user_gender_id*shop_star_level"] = tmp_train_df["user_gender_id"] * (1+tmp_train_df["shop_star_level"])


cols_tool.fit(train_df)

categorical_one_hot_cols = cols_tool.get_onehoted_cols("categorical")
listype_one_hot_cols = cols_tool.get_onehoted_cols("listype")

index_col = "instance_id"


categorical_cols = cols_tool.get_raw_categorical_cols()
listype_cols = cols_tool.get_raw_listype_cols()
numerical_cols = cols_tool.get_raw_numerical_cols()
target_col = cols_tool.get_raw_target_col()


print("org dataset = %s" % len(tmp_train_df))
print(tmp_train_df["day"].value_counts())


# tmp_train_df = featProc.balance_pos_neg_sample(tmp_train_df, (1,10))
# print "sampling dataset =", len(tmp_train_df)
# print tmp_train_df["day"].value_counts()

# 抽样
# tmp_train_df = tmp_train_df.sample(n=10000, random_state=666)
# print "sampling dataset =", len(tmp_train_df)

print(tmp_train_df["day"].value_counts())

#测试版
valid_df = tmp_train_df.loc[tmp_train_df["day"]==6]
tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]<6]
# tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]>0]
res_df=valid_df.copy()

# #线上验证版
# valid_df = tmp_train_df.loc[tmp_train_df["day"]==6]
# res_df = tmp_train_df.loc[tmp_train_df["day"]==7]
# tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]<=6]

# print("res dataset = %s" % len(res_df))
# print("valid dateset = %s" % len(valid_df))
# print("train dateset = %s" % len(tmp_train_df))

# tmp_train_df = tmp_train_df[on_cols]
# valid_df = valid_df[on_cols]


# on_cols = [target_col]+numerical_cols+categorical_one_hot_cols+listype_one_hot_cols
# # on_cols = [target_col]+numerical_cols
# # +categorical_one_hot_cols+listype_one_hot_cols
# train_lls, test_lls = train_and_test_lr(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("LR %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))



# on_cols = [target_col]+categorical_cols+numerical_cols
# # on_cols = [target_col]+select_best_features(tmp_train_df[on_cols], 0.8)
# train_lls, test_lls = train_and_test_randomforest(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("XGBoost %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))



# on_cols = [target_col]+categorical_cols+numerical_cols
# on_cols = [target_col]+select_best_features(tmp_train_df[on_cols], 0.95)
# train_lls, test_lls = train_and_test_randomforest(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("XGBoost %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))




# +listype_one_hot_cols
# on_cols = ffea(on_cols)



# on_cols = [target_col]+categorical_cols+numerical_cols+listype_one_hot_cols
# train_lls, test_lls = train_and_test_randomforest(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("RandomForest %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))

# on_cols = [target_col]+categorical_cols+numerical_cols+listype_one_hot_cols
# train_lls, test_lls = train_and_test_gbdt(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("GBDT %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))


# train_lls, test_lls = train_and_test_combine(tmp_train_df[on_cols], valid_df[on_cols])
# print "Combine TrainLogLoss = %s | TestLogLoss = %s" % (train_lls, test_lls)

# on_cols = [target_col]+numerical_cols+categorical_one_hot_cols+listype_one_hot_cols
# train_lls, test_lls = train_and_test_FFM(tmp_train_df, valid_df)
# print "FFM TrainLogLoss = %s | TestLogLoss = %s" % (train_lls, test_lls)

# train_lls, test_lls = train_and_test_FFM(tmp_train_df, valid_df, res_df)
# print("LR %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))




org dataset = 496509
0    78268
3    71199
1    70931
2    68387
4    68318
5    63614
6    57421
7    18371
Name: day, dtype: int64
0    78268
3    71199
1    70931
2    68387
4    68318
5    63614
6    57421
7    18371
Name: day, dtype: int64


In [15]:
tmp_train_df = train_df.copy()
valid_df = gen_valid_set(tmp_train_df)
# valid_df = gen_valid_set_FULL(tmp_train_df)

tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]<6]
res_df=valid_df.copy()


print len(tmp_train_df),  len(valid_df)


categorical_one_hot_cols = cols_tool.get_onehoted_cols("categorical", tmp_train_df)
listype_one_hot_cols = cols_tool.get_onehoted_cols("listype", tmp_train_df)

index_col = "instance_id"

categorical_cols = cols_tool.get_raw_categorical_cols()
listype_cols = cols_tool.get_raw_listype_cols()
numerical_cols = cols_tool.get_raw_numerical_cols()
target_col = cols_tool.get_raw_target_col()

NameError: name 'gen_valid_set' is not defined

In [ ]:
print len(valid_df)
print valid_df["is_trade"].value_counts()

In [ ]:
on_cols = [target_col]+numerical_cols+categorical_one_hot_cols#+listype_one_hot_cols
# print on_cols
train_lls, test_lls = train_and_test_lr(tmp_train_df[on_cols], valid_df[on_cols], None)
print("LR TrainLogLoss = %s | TestLogLoss = %s" % (train_lls, test_lls))


In [ ]:
X_test = valid_df.copy()
del X_test['is_trade']
y_test = valid_df.copy()['is_trade']

import random
from sklearn.model_selection import train_test_split

for i in xrange(1000000):
    X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=i)
    zero = []
    for j in xrange(len(y_val_a)):
        f = float('0.0000000000000%s' % random.randint(1,99))
        zero.append(f)
    lls = log_loss(y_val_a, zero)
    llsdiff = abs(0.31284 - lls)
    y_val_a = pd.DataFrame(y_val_a)
    l = float(len(y_val_a))
    
    if llsdiff < 0.0001:
        print i, "[HIT]", llsdiff, lls, len(y_val_a[y_val_a["is_trade"] == 0])/l, len(y_val_a[y_val_a["is_trade"] == 1])/l
    else:
        print i, llsdiff, lls, len(y_val_a[y_val_a["is_trade"] == 0])/l, len(y_val_a[y_val_a["is_trade"] == 1])/l
        


In [ ]:

t = pd.read_table('/Users/yuhua/res1.csv',sep=' ')
log_loss(a, t["predicted_score"])


In [ ]:
print cols_tool.numerical

In [28]:
%run ffmyh.py
%run FeatureProcess.py

fpfp = FeatureProcess(target=target, categorical=categorical, numerical=numerical, listype = listype)
# for n in numerical:
#     fpfp.addLayeringOrgKeys(n)
#一定要fit一个全集
fpfp.fit(train_df)

fpfp.toFFMData(tmp_train_df, "one_hot-train.txt")
fpfp.toFFMData(valid_df, "one_hot-valid.txt")
# model = FFM(fpfp,
#             reg_param = 0.00001,
#             k = 4,
#             iter_max = 500,
#             learing_rate = 0.01,
#             threads = 7,
#             auto_stop = False,
#             quiet = False)


    
# model.fit(tmp_train_df, valid_df)

# train_lls = log_loss(train_df["is_trade"],model.predict_proba(train_df))
# test_lls = log_loss(test_df["is_trade"],model.predict_proba(test_df))

# print train_lls, test_lls


fiting ... 


KeyboardInterrupt: 

- | XGboost | 增减 | LR |增减
- | :-: | :-: | :-: | :-:
原始特征 | 0.0829350458274 | / | 0.0829428179401 | /
原始特征+时间 | 0.0828004994761 | +++ | 0.082869098617 | +
同上+按天计算交易率 | 0.0828218434014 | - | 0.0827914564667 | ++
同上+predict_prop_cate相似度 | 0.082778758021 | ++ | 0.082798919764 | --
同上+cate prop丰富度 | 0.0828356414143 | -- | 0.0828009736655 | -
同上+cate/prop list OneHot | 0.0828356414143 | / | 0.0827979171146 | +

In [ ]:
LogisticRegression()

In [ ]:
help(LogisticRegression)

In [ ]:
print tmp_train_df.columns

In [ ]:
help(SelectKBest(chi2,k=2).fit_transform)